# 第10章: 事前学習済み言語モデル（GPT型）

本章では、GPT型（Transformerのデコーダ型）の事前学習済みモデルを利用して、言語生成、評判分析器（ポジネガ分類器）の構築、ファインチューニング、強化学習などに取り組む。

## 90. 次単語予測

“The movie was full of"に続くトークン（トークン列ではなく一つのトークンであることに注意せよ）として適切なもの上位10個と、その確率（尤度）を求めよ。ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def get_next_token_predictions(prompt_text: str, model_name: str = "gpt2", top_k: int = 10):
    """
    指定されたプロンプトに続く次のトークンの上位K個とその確率を予測します。
    また、プロンプトがどのようにトークン化されたかを表示します。

    Args:
        prompt_text (str): 入力プロンプト文字列。
        model_name (str, optional): 使用する事前学習済みモデルの名前。
                                     デフォルトは "gpt2"。
        top_k (int, optional): 取得する上位トークンの数。デフォルトは 10。

    Returns:
        list: 予測されたトークンとその確率のリスト。
              各要素は (トークン文字列, 確率) のタプル。
    """
    try:
        # 1. モデルとトークナイザーのロード
        print(f"Loading model and tokenizer for '{model_name}'...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)

        # モデルを評価モードに設定 (勾配計算をオフにするなど)
        model.eval()

        # 2. プロンプトのトークン化
        print(f"\n--- Tokenizing Prompt: \"{prompt_text}\" ---")
        # トークナイザーによっては、プロンプトの先頭にスペースが必要な場合があるため、
        # GPT-2のようなモデルではそのままで良いが、モデルに応じて調整することも考慮
        inputs = tokenizer(prompt_text, return_tensors="pt")
        input_ids = inputs["input_ids"]

        print(f"Input IDs: {input_ids.tolist()[0]}")
        tokens = [tokenizer.decode([token_id]) for token_id in input_ids[0].tolist()]
        # トークンによっては、デコード時に不要なスペースが含まれることがあるため、必要に応じて調整
        # 例: tokens = [tokenizer.convert_ids_to_tokens(token_id) for token_id in input_ids[0].tolist()]
        print(f"Tokens: {tokens}")
        print("--------------------------------------")

        # 3. 次のトークンの予測
        with torch.no_grad(): # 勾配計算を行わないコンテキスト
            outputs = model(input_ids)
            # outputs.logits の形状は (batch_size, sequence_length, vocab_size)
            # 次のトークンの予測なので、最後のトークン位置のlogitsを使用
            next_token_logits = outputs.logits[0, -1, :]

        # 4. Logitsを確率に変換 (Softmax)
        probabilities = torch.softmax(next_token_logits, dim=-1)

        # 5. 上位K個のトークンとその確率を取得
        top_k_probabilities, top_k_indices = torch.topk(probabilities, top_k)

        results = []
        print(f"\n--- Top {top_k} next token predictions for \"{prompt_text}\" ---")
        for i in range(top_k):
            token_id = top_k_indices[i].item()
            token_probability = top_k_probabilities[i].item()
            # token_idを文字列にデコード
            # decodeメソッドはリスト形式のIDも受け付ける
            token_string = tokenizer.decode([token_id])
            results.append((token_string, token_probability))
            print(f"{i+1}. Token: \"{token_string}\" (ID: {token_id}), Probability: {token_probability:.6f}")
        print("----------------------------------------------------")
        return results

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

if __name__ == "__main__":
    prompt = "The movie was full of"
    # GPT-2モデルを使用します。他のモデル (例: "gpt2-medium", "gpt2-large", "EleutherAI/gpt-neo-1.3B" など) も試せます。
    # 大きなモデルほど、より高品質な予測が期待できますが、計算リソースも多く必要とします。
    predicted_tokens = get_next_token_predictions(prompt, model_name="gpt2", top_k=10)

    # (オプション) 他のGPT系モデルを試す場合
    # print("\nTrying with gpt2-medium (may take longer to download/run):")
    # predicted_tokens_medium = get_next_token_predictions(prompt, model_name="gpt2-medium", top_k=10)

Loading model and tokenizer for 'gpt2'...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


--- Tokenizing Prompt: "The movie was full of" ---
Input IDs: [464, 3807, 373, 1336, 286]
Tokens: ['The', ' movie', ' was', ' full', ' of']
--------------------------------------

--- Top 10 next token predictions for "The movie was full of" ---
1. Token: " jokes" (ID: 14532), Probability: 0.021892
2. Token: " great" (ID: 1049), Probability: 0.018644
3. Token: " laughs" (ID: 22051), Probability: 0.011524
4. Token: " bad" (ID: 2089), Probability: 0.010874
5. Token: " surprises" (ID: 24072), Probability: 0.010667
6. Token: " references" (ID: 10288), Probability: 0.010528
7. Token: " fun" (ID: 1257), Probability: 0.009992
8. Token: " humor" (ID: 14733), Probability: 0.007415
9. Token: " "" (ID: 366), Probability: 0.007408
10. Token: " the" (ID: 262), Probability: 0.006709
----------------------------------------------------


## 91. 続きのテキストの予測

“The movie was full of"に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_text_with_varying_strategies(prompt_text: str, model_name: str = "gpt2"):
    """
    指定されたプロンプトに対し、異なるデコーディング戦略やパラメータを用いて
    複数のテキストを生成し、その結果を表示します。

    Args:
        prompt_text (str): 入力プロンプト文字列。
        model_name (str, optional): 使用する事前学習済みモデルの名前。
                                     デフォルトは "gpt2"。
    """
    try:
        # 1. モデルとトークナイザーのロード
        print(f"Loading model and tokenizer for '{model_name}'...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)

        # トークナイザーにパディングトークンが設定されていない場合、EOSトークンを設定
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = model.config.eos_token_id


        # モデルを評価モードに設定
        model.eval()

        # プロンプトをトークンIDに変換
        # attention_maskも一緒に渡すことで、パディングトークンを無視できる
        inputs = tokenizer(prompt_text, return_tensors="pt", padding=True)
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]


        # 生成するテキストの共通パラメータ
        # max_lengthはプロンプトの長さを考慮して設定。ここではプロンプトに加えて約30トークン生成。
        # プロンプトのトークン数を取得
        prompt_num_tokens = input_ids.shape[-1]
        max_new_tokens = 30 # 新しく生成するトークンの最大数
        max_length = prompt_num_tokens + max_new_tokens

        num_return_sequences = 2 # 各設定で生成するシーケンスの数

        print(f"\n--- Prompt: \"{prompt_text}\" ---")
        print(f"(Generating up to {max_new_tokens} new tokens beyond the prompt)")

        # --- 1. Greedy Search ---
        # 各ステップで最も確率の高いトークンを選択。決定的で高速だが、単調になりやすい。
        print("\n--- Strategy 1: Greedy Search ---")
        greedy_outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_return_sequences=1, # Greedy Searchは通常1つのシーケンスを返す
            do_sample=False # サンプリングを無効化
        )
        for i, output in enumerate(greedy_outputs):
            decoded_output = tokenizer.decode(output, skip_special_tokens=True)
            print(f"Greedy Output {i+1}: {decoded_output}")

        # --- 2. Beam Search ---
        # 複数の候補（ビーム）を保持し、全体として最も良いシーケンスを探す。
        # Greedyより高品質な傾向があるが、計算コストが高く、多様性に欠けることも。
        num_beams = 5
        print(f"\n--- Strategy 2: Beam Search (num_beams={num_beams}) ---")
        beam_outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=max_length,
            num_beams=num_beams,
            num_return_sequences=num_return_sequences,
            early_stopping=True, # 全てのビームがEOSに到達したら停止
            no_repeat_ngram_size=2 # 同じn-gramの繰り返しを避ける (例: "I think I think")
        )
        for i, output in enumerate(beam_outputs):
            decoded_output = tokenizer.decode(output, skip_special_tokens=True)
            print(f"Beam Output {i+1} (num_beams={num_beams}): {decoded_output}")

        # --- 3. Sampling with varying Temperature ---
        # 次のトークンを確率分布に基づいてランダムに選択。
        # Temperature: 値が低いと高確率のトークンが選ばれやすく（決定的）、高いと低確率のトークンも選ばれ（ランダム性が増す）。
        temperatures = [0.7, 1.0, 1.5]
        print("\n--- Strategy 3: Sampling with varying Temperature (top_k=50) ---")
        for temp in temperatures:
            print(f"  - Temperature = {temp}")
            sampling_outputs_temp = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                do_sample=True, # サンプリングを有効化
                temperature=temp,
                top_k=50, # 上位50件のトークンからサンプリング
                num_return_sequences=num_return_sequences,
                no_repeat_ngram_size=2
            )
            for i, output in enumerate(sampling_outputs_temp):
                decoded_output = tokenizer.decode(output, skip_special_tokens=True)
                print(f"    Output {i+1}: {decoded_output}")

        # --- 4. Sampling with Top-K ---
        # Top-K Sampling: 確率上位K個のトークンのみをサンプリング対象とする。
        top_k_values = [10, 50] # top_k=0で無効化 (通常のtemperature samplingになる)
        print(f"\n--- Strategy 4: Sampling with Top-K (Temperature=1.0) ---")
        for k_val in top_k_values:
            print(f"  - Top-K = {k_val}")
            sampling_outputs_top_k = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                do_sample=True,
                temperature=1.0, # Temperatureは固定してTop-Kの効果を見る
                top_k=k_val,
                num_return_sequences=num_return_sequences,
                no_repeat_ngram_size=2
            )
            for i, output in enumerate(sampling_outputs_top_k):
                decoded_output = tokenizer.decode(output, skip_special_tokens=True)
                print(f"    Output {i+1}: {decoded_output}")

        # --- 5. Sampling with Top-P (Nucleus Sampling) ---
        # Top-P Sampling: 確率の累積がPを超えるまでの最小のトークンセットをサンプリング対象とする。
        # 候補の数を動的に調整する。
        top_p_values = [0.90, 0.95] # top_p=1.0で無効化
        print(f"\n--- Strategy 5: Sampling with Top-P (Nucleus Sampling, Temperature=1.0) ---")
        for p_val in top_p_values:
            print(f"  - Top-P = {p_val}")
            # top_k=0 を設定して、top_pサンプリングを確実に有効にする
            sampling_outputs_top_p = model.generate(
                input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                do_sample=True,
                temperature=1.0,
                top_p=p_val,
                top_k=0, # Top-KとTop-Pは通常どちらかを主に使用するため、Top-Kは無効化
                num_return_sequences=num_return_sequences,
                no_repeat_ngram_size=2
            )
            for i, output in enumerate(sampling_outputs_top_p):
                decoded_output = tokenizer.decode(output, skip_special_tokens=True)
                print(f"    Output {i+1}: {decoded_output}")
        print("----------------------------------------------------")

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    prompt = "The movie was full of"
    # "gpt2" は比較的小さなモデルです。より大きなモデル (例: "gpt2-medium", "EleutherAI/gpt-neo-1.3B") も試せますが、
    # リソースと時間が必要になります。
    generate_text_with_varying_strategies(prompt, model_name="gpt2")

Loading model and tokenizer for 'gpt2'...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



--- Prompt: "The movie was full of" ---
(Generating up to 30 new tokens beyond the prompt)

--- Strategy 1: Greedy Search ---


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Greedy Output 1: The movie was full of jokes and jokes about how the movie was a joke. It was a joke about how the movie was a joke. It was a joke about how the

--- Strategy 2: Beam Search (num_beams=5) ---


...

    Output 1: The movie was full of the kinds of people who were really excited to see it, including myself. I was like, 'Holy shit, this movie is really cool.' And
    Output 2: The movie was full of humor, and it was a great way to show off the characters. It made a lot of people laugh.

What did you think about how
  - Top-K = 50


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


    Output 1: The movie was full of heartache, and many of its key characters, namely Shizune and Yuki, turned on their own.

According to sources close to
    Output 2: The movie was full of many jokes because it reminded me of those with a certain feel about them. They're all in the same movie, and that's what it really comes

--- Strategy 5: Sampling with Top-P (Nucleus Sampling, Temperature=1.0) ---
  - Top-P = 0.9


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


    Output 1: The movie was full of uncomfortable references to "SNL," with long pauses to swap out to Pizarro, a recurring character in the original series who died a few years
    Output 2: The movie was full of talk about "lethal weapons." But similar characters in a very real movie, with some notable differences, appear more frequently in serials, notably these world
  - Top-P = 0.95
    Output 1: The movie was full of unexpected twists and has been starred in a host of other blockbuster films.

But some of the actors' star power stems from the chemistry between the
    Output 2: The movie was full of black humour to begin with and kept going. As you have seen in the story I am a fan of Blade Runner but these are lesser known movies that
----------------------------------------------------


## 92. 予測されたテキストの確率を計算

“The movie was full of"に続くテキストを予測し、生成された各単語の尤度を表示せよ（生成されるテキストが長いと出力が読みにくくなるので、適当な長さで生成を打ち切るとよい）。

In [ ]:
import torch
import torch.nn.functional as F # Softmax関数を利用するためにインポート
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_text_with_token_probabilities(prompt_text: str, model_name: str = "gpt2", max_new_tokens: int = 10):
    """
    指定されたプロンプトに続くテキストを生成し、各生成トークンの尤度（確率）を表示します。
    Greedy Searchを使用します。

    Args:
        prompt_text (str): 入力プロンプト文字列。
        model_name (str, optional): 使用する事前学習済みモデルの名前。デフォルトは "gpt2"。
        max_new_tokens (int, optional): 新しく生成するトークンの最大数。デフォルトは 10。
    """
    try:
        # 1. モデルとトークナイザーのロード
        print(f"Loading model and tokenizer for '{model_name}'...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)

        # モデルを評価モードに設定
        model.eval()

        # パディングトークンが未設定の場合、EOSトークンをパディングトークンとして設定
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = model.config.eos_token_id

        # 2. プロンプトのトークン化
        inputs = tokenizer(prompt_text, return_tensors="pt")
        input_ids = inputs["input_ids"]
        # attention_mask は generate メソッド内で input_ids に基づいて自動生成されることが多いですが、
        # 明示的に渡すことも可能です。ここでは省略し、generateメソッドのデフォルトの振る舞いに任せます。

        print(f"\n--- Prompt: \"{prompt_text}\" ---")

        # 3. テキスト生成とスコアの取得
        # output_scores=True: 各生成ステップでの全語彙に対するスコア(logits)を返す
        # return_dict_in_generate=True: 出力を辞書形式で受け取り、'sequences'や'scores'にアクセスしやすくする
        # do_sample=False: Greedy Searchを行う (最も確率の高いトークンを選択)
        with torch.no_grad(): # 勾配計算を無効化
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                output_scores=True,
                return_dict_in_generate=True,
                do_sample=False # Greedy search
                # --- サンプリングで試す場合の例 ---
                # do_sample=True,
                # temperature=0.7,
                # top_k=50,
            )

        # 生成されたシーケンス全体 (プロンプト部分も含む)
        # outputs.sequences の形状は (batch_size, sequence_length)
        # ここでは batch_size = 1 を想定
        generated_sequence_ids = outputs.sequences[0]

        # 各生成ステップでのロジットのタプル
        # outputs.scores の各要素は、そのステップで生成されるトークンの全語彙に対するロジット
        # タプルの長さは max_new_tokens と同じ
        # 各ロジットテンソルの形状は (batch_size, vocab_size)
        step_scores = outputs.scores

        # プロンプト部分の長さを取得
        prompt_length = input_ids.shape[1]

        print(f"\n--- Generated Text with Token Probabilities (Strategy: Greedy Search, Max New Tokens: {max_new_tokens}) ---")

        generated_tokens_with_probs = []

        for i in range(len(step_scores)): # 生成された各ステップについてループ
            # i番目のステップで生成されたトークンのID (プロンプトの次から数えてi番目)
            # generated_sequence_ids の (prompt_length + i) 番目の要素
            actual_generated_token_id = generated_sequence_ids[prompt_length + i].item()

            # i番目のステップのロジットを取得 (batch_size=1を想定)
            # step_scores[i] の形状は (1, vocab_size)
            logits_for_this_step = step_scores[i][0] # batch_size次元を削除

            # ロジットを確率に変換
            probabilities_for_this_step = F.softmax(logits_for_this_step, dim=-1)

            # 実際に生成されたトークンの確率を取得
            probability_of_actual_token = probabilities_for_this_step[actual_generated_token_id].item()

            # トークンIDを文字列にデコード
            token_string = tokenizer.decode([actual_generated_token_id])

            generated_tokens_with_probs.append({
                "token_str": token_string,
                "token_id": actual_generated_token_id,
                "probability": probability_of_actual_token
            })
            print(f"Step {i+1}: Token: \"{token_string}\" (ID: {actual_generated_token_id}), Probability: {probability_of_actual_token:.4f}")

        # 全体の生成テキストを表示
        full_generated_text = tokenizer.decode(generated_sequence_ids, skip_special_tokens=True)
        print(f"\nFull generated text: {full_generated_text}")
        print("----------------------------------------------------")

        return generated_tokens_with_probs

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()
        return []

if __name__ == "__main__":
    prompt = "The movie was full of"
    # 新しく生成するトークン数を10に設定
    generated_info = generate_text_with_token_probabilities(prompt, model_name="gpt2", max_new_tokens=10)

    # (オプション) サンプリング戦略で試す場合
    # その場合は、generate_text_with_token_probabilities 関数の model.generate の呼び出し部分で
    # do_sample=True とし、必要に応じて temperature, top_k, top_p などのパラメータを設定してください。
    # サンプリングの場合、表示される確率は「そのステップでサンプリングによって選ばれたトークンが元々持っていた確率」となります。

Loading model and tokenizer for 'gpt2'...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--- Prompt: "The movie was full of" ---

--- Generated Text with Token Probabilities (Strategy: Greedy Search, Max New Tokens: 10) ---
Step 1: Token: " jokes" (ID: 14532), Probability: 0.0219
Step 2: Token: " and" (ID: 290), Probability: 0.2892
Step 3: Token: " jokes" (ID: 14532), Probability: 0.0985
Step 4: Token: " about" (ID: 546), Probability: 0.2056
Step 5: Token: " how" (ID: 703), Probability: 0.0997
Step 6: Token: " the" (ID: 262), Probability: 0.0846
Step 7: Token: " movie" (ID: 3807), Probability: 0.0364
Step 8: Token: " was" (ID: 373), Probability: 0.2963
Step 9: Token: " a" (ID: 257), Probability: 0.0677
Step 10: Token: " joke" (ID: 9707), Probability: 0.1735

Full generated text: The movie was full of jokes and jokes about how the movie was a joke
----------------------------------------------------


## 93. パープレキシティ

適当な文を準備して、事前学習済み言語モデルでパープレキシティを測定せよ。例えば、

+ The movie was full of surprises
+ The movies were full of surprises
+ The movie were full of surprises
+ The movies was full of surprises

の4文に対して、パープレキシティを測定して観察せよ（最後の2つの文は故意に文法的な間違いを入れた）。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import math # math.exp を使う場合

def calculate_perplexity_for_sentence(text: str, model, tokenizer, device="cpu"):
    """
    与えられた単一の文に対してパープレキシティを計算します。

    Args:
        text (str): パープレキシティを計算するテキスト文字列。
        model (PreTrainedModel): 事前学習済み言語モデル。
        tokenizer (PreTrainedTokenizer): 対応するトークナイザー。
        device (str): 計算に使用するデバイス ("cpu" または "cuda")。

    Returns:
        tuple: (パープレキシティ, 平均負の対数尤度)
               計算に失敗した場合は (float('inf'), float('inf'))
    """
    model.to(device) # モデルを計算デバイスに移動

    # テキストをトークン化
    # Hugging FaceのモデルでPPLを計算する際は、入力テキスト全体をinput_idsとし、
    # 同じものをlabelsとして渡すのが一般的です。モデル内部で適切にシフトして損失を計算します。
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    # attention_mask = inputs.attention_mask.to(device) # labelsを指定する場合、通常attention_maskは必須ではない

    # 損失（平均NLL）を計算
    with torch.no_grad(): # 勾配計算は不要
        outputs = model(input_ids, labels=input_ids)
        # outputs.loss は、シーケンス全体のトークンごとの平均クロスエントロピー損失（負の対数尤度）
        neg_log_likelihood = outputs.loss

    if neg_log_likelihood is None or torch.isnan(neg_log_likelihood) or torch.isinf(neg_log_likelihood):
        print(f"Warning: Could not compute valid NLL for '{text}'. Assigning high PPL.")
        return float('inf'), float('inf')

    # パープレキシティを計算: PPL = exp(NLL)
    # neg_log_likelihood はスカラテンソルなので .item() でPythonの数値に変換可能
    ppl = torch.exp(neg_log_likelihood).item()
    # または ppl = math.exp(neg_log_likelihood.item())

    return ppl, neg_log_likelihood.item()

if __name__ == "__main__":
    # 使用するモデルを選択 (例: "gpt2", "gpt2-medium", "distilgpt2" など)
    # "gpt2" は比較的小さく、ダウンロードも容易です。
    model_name = "gpt2"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    print(f"\nLoading model and tokenizer for '{model_name}'...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)
        model.eval() # 評価モードに設定
    except Exception as e:
        print(f"Error loading model or tokenizer: {e}")
        exit()

    sentences_to_evaluate = [
        "The movie was full of surprises",    # 文法的に正しい (単数)
        "The movies were full of surprises",  # 文法的に正しい (複数)
        "The movie were full of surprises",   # 文法誤り (単数主語 + 複数動詞)
        "The movies was full of surprises"    # 文法誤り (複数主語 + 単数動詞)
    ]

    print(f"\n--- Calculating Perplexity using '{model_name}' ---")
    results = []
    for sentence in sentences_to_evaluate:
        ppl, nll = calculate_perplexity_for_sentence(sentence, model, tokenizer, device)
        results.append({
            "sentence": sentence,
            "nll": nll,
            "perplexity": ppl
        })
        print(f"\nSentence: \"{sentence}\"")
        if math.isinf(ppl):
            print(f"  Negative Log Likelihood (NLL): N/A")
            print(f"  Perplexity (PPL): Infinite / Not Calculable")
        else:
            print(f"  Negative Log Likelihood (NLL): {nll:.4f}")
            print(f"  Perplexity (PPL): {ppl:.4f}")

    print("\n\n--- Observation Summary ---")
    print("Perplexity (PPL) measures how well a probability distribution or probability model predicts a sample.")
    print("A lower PPL indicates the model is less 'surprised' by the sentence, meaning it considers the sentence more probable or natural.")
    print("Generally, we expect:")
    print("  - Grammatically correct sentences to have lower PPL than incorrect ones.")
    print("  - More common or plausible phrasings to have lower PPL.")

    # 結果をPPLでソートして表示
    if any(not math.isinf(res["perplexity"]) for res in results):
        sorted_results = sorted(results, key=lambda x: x["perplexity"])
        print("\nSentences sorted by Perplexity (lower is generally better):")
        for res in sorted_results:
            if math.isinf(res["perplexity"]):
                print(f"  PPL: Infinite - NLL: N/A      - \"{res['sentence']}\"")
            else:
                print(f"  PPL: {res['perplexity']:.2f}     - NLL: {res['nll']:.2f} - \"{res['sentence']}\"")
    else:
        print("\nCould not calculate PPL for any sentence.")

    print("\n--- Expected Outcome based on the input sentences ---")
    print("1. \"The movie was full of surprises\" (Correct Singular): Should have a relatively low PPL.")
    print("2. \"The movies were full of surprises\" (Correct Plural): Should also have a relatively low PPL, possibly similar to the singular correct version.")
    print("3. \"The movie were full of surprises\" (Incorrect - singular noun, plural verb): Should have a higher PPL than (1).")
    print("4. \"The movies was full of surprises\" (Incorrect - plural noun, singular verb): Should have a higher PPL than (2).")
    print("The two grammatically incorrect sentences are expected to show significantly higher PPL values, reflecting the model's 'surprise' or difficulty in predicting such ungrammatical sequences.")

Using device: cuda

Loading model and tokenizer for 'gpt2'...

--- Calculating Perplexity using 'gpt2' ---


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.



Sentence: "The movie was full of surprises"
  Negative Log Likelihood (NLL): 4.5987
  Perplexity (PPL): 99.3545

Sentence: "The movies were full of surprises"
  Negative Log Likelihood (NLL): 4.8401
  Perplexity (PPL): 126.4840

Sentence: "The movie were full of surprises"
  Negative Log Likelihood (NLL): 5.6308
  Perplexity (PPL): 278.8832

Sentence: "The movies was full of surprises"
  Negative Log Likelihood (NLL): 5.6156
  Perplexity (PPL): 274.6661


--- Observation Summary ---
Perplexity (PPL) measures how well a probability distribution or probability model predicts a sample.
A lower PPL indicates the model is less 'surprised' by the sentence, meaning it considers the sentence more probable or natural.
Generally, we expect:
  - Grammatically correct sentences to have lower PPL than incorrect ones.
  - More common or plausible phrasings to have lower PPL.

Sentences sorted by Perplexity (lower is generally better):
  PPL: 99.35     - NLL: 4.60 - "The movie was full of surprises"

## 94. チャットテンプレート

"What do you call a sweet eaten after dinner?"という問いかけに対する応答を生成するため、チャットテンプレートを適用し、言語モデルに与えるべきプロンプトを作成せよ。また、そのプロンプトに対する応答を生成し、表示せよ。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_chat_response(question: str, model_name: str = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"):
    """
    チャットテンプレートを適用してプロンプトを作成し、言語モデルからの応答を生成します。

    Args:
        question (str): ユーザーからの問いかけ。
        model_name (str, optional): 使用する対話型モデルの名前。
                                     デフォルトは "TinyLlama/TinyLlama-1.1B-Chat-v1.0"。
    """
    try:
        print(f"Loading model and tokenizer for '{model_name}'...")
        # 1. モデルとトークナイザーのロード
        # device_map="auto" を使うと、利用可能なGPUに自動でモデルを割り振ります。
        # CPUのみの場合は device_map="cpu" と同様の動作になります。
        # 量子化されたモデルを使う場合は、追加のライブラリや設定が必要なことがあります。
        # 例: model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

        device = model.device # モデルがロードされたデバイスを取得
        print(f"Model loaded on device: {device}")
        model.eval()  # 評価モードに設定

        # 2. 会話履歴の作成
        # システムプロンプトはモデルの振る舞いを指示するのに役立ちます (オプション)
        messages = [
            {"role": "system", "content": "You are a helpful and knowledgeable assistant."},
            {"role": "user", "content": question}
        ]

        # 3. チャットテンプレートの適用
        # tokenize=False: プロンプトを文字列として取得
        # add_generation_prompt=True: モデルが応答を続けるための接尾辞をプロンプトに追加
        # (例: アシスタントの応答開始を示す "<|assistant|>\n" など)
        prompt_string = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        print("\n--- Generated Prompt using Chat Template ---")
        print(f"Template applied by tokenizer for '{model_name}':")
        print(prompt_string)
        print("------------------------------------------")

        # 4. プロンプトをトークン化 (文字列プロンプトを使用する場合)
        # apply_chat_templateでtokenize=Trueにすれば、このステップは不要で直接トークンIDが得られます。
        inputs = tokenizer(prompt_string, return_tensors="pt").to(device)
        input_ids = inputs.input_ids
        # attention_maskも同様に inputs.attention_mask で取得できますが、
        # 単一シーケンスの生成では model.generate が内部で処理してくれることが多いです。

        # 5. 応答の生成
        # pad_token_idをeos_token_idに設定するのは一般的な対処法です。
        # 特にバッチ処理や左パディングが必要なモデルで重要になります。
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = model.config.eos_token_id

        print("\nGenerating response...")
        with torch.no_grad():
            # max_new_tokens: 新しく生成するトークンの最大数
            # temperature: サンプリングの際のランダム性を制御 (低いと決定的、高いと多様)
            # top_k, top_p: サンプリングの候補を絞り込む手法
            generation_output = model.generate(
                input_ids,
                max_new_tokens=60,       # 応答の長さを適度に制限
                do_sample=True,          # サンプリングを有効化
                temperature=0.7,
                top_k=50,
                top_p=0.95,
                pad_token_id=tokenizer.eos_token_id # パディングトークンIDを指定
            )

        # 生成された応答部分のみをデコード
        # generation_output[0] はプロンプトを含む全体のシーケンス
        # input_ids.shape[-1] はプロンプトのトークン長
        response_ids = generation_output[0][input_ids.shape[-1]:]
        response_text = tokenizer.decode(response_ids, skip_special_tokens=True)

        print("\n--- Generated Response ---")
        print(response_text)
        print("--------------------------")

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    user_question = "What do you call a sweet eaten after dinner?"
    generate_chat_response(user_question)

    # (オプション) 他のチャットモデルで試す場合 (モデルサイズやアクセス権に注意)
    # モデルによっては、チャットテンプレートの形式が異なる場合がありますが、
    # tokenizer.apply_chat_template がそれを吸収してくれます。
    # 例:
    # generate_chat_response(user_question, model_name="mistralai/Mistral-7B-Instruct-v0.2")
    # generate_chat_response(user_question, model_name="HuggingFaceH4/zephyr-7b-beta")

Loading model and tokenizer for 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'...


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded on device: cuda:0

--- Generated Prompt using Chat Template ---
Template applied by tokenizer for 'TinyLlama/TinyLlama-1.1B-Chat-v1.0':
<|system|>
You are a helpful and knowledgeable assistant.</s>
<|user|>
What do you call a sweet eaten after dinner?</s>
<|assistant|>

------------------------------------------

Generating response...

--- Generated Response ---
A post-dinner treat or dessert.
--------------------------


## 95. マルチターンのチャット

問題94で生成された応答に対して、追加で"Please give me the plural form of the word with its spelling in reverse order."と問いかけたときの応答を生成・表示せよ。また、その時に言語モデルに与えるプロンプトを確認せよ。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def generate_follow_up_chat_response(
    previous_interactions: list,
    follow_up_question: str,
    model_name: str = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
):
    """
    既存の会話履歴に続けて、新たな質問に対する応答を生成します。

    Args:
        previous_interactions (list): これまでの会話のリスト。各要素は
                                      {"role": "user/assistant/system", "content": "message"} の形式。
        follow_up_question (str): 新たなユーザーからの問いかけ。
        model_name (str, optional): 使用する対話型モデルの名前。
                                     デフォルトは "TinyLlama/TinyLlama-1.1B-Chat-v1.0"。
    """
    try:
        print(f"Loading model and tokenizer for '{model_name}' (if not already cached)...")
        # 1. モデルとトークナイザーのロード
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

        device = model.device
        print(f"Model is on device: {device}")
        model.eval()

        # 2. 会話履歴の更新
        # 新しいユーザーの質問を会話履歴に追加
        current_messages = previous_interactions + [{"role": "user", "content": follow_up_question}]

        # 3. チャットテンプレートの適用
        prompt_string = tokenizer.apply_chat_template(
            current_messages,
            tokenize=False,
            add_generation_prompt=True  # アシスタントの応答開始を促す
        )

        print("\n--- Generated Prompt for the Follow-up Question (including history) ---")
        print(prompt_string)
        print("---------------------------------------------------------------------")

        # 4. プロンプトをトークン化
        inputs = tokenizer(prompt_string, return_tensors="pt").to(device)
        input_ids = inputs.input_ids

        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            model.config.pad_token_id = model.config.eos_token_id

        # 5. 応答の生成
        print("\nGenerating response to the follow-up question...")
        with torch.no_grad():
            generation_output = model.generate(
                input_ids,
                max_new_tokens=80,        # 少し長めの応答も許容できるように調整
                do_sample=True,
                temperature=0.6,          # 少しだけ決定論的な方向に (前回0.7と仮定)
                top_k=50,
                top_p=0.90,               # サンプリング候補を少し絞る
                pad_token_id=tokenizer.eos_token_id
            )

        # 生成された応答部分のみをデコード
        response_ids = generation_output[0][input_ids.shape[-1]:]
        response_text = tokenizer.decode(response_ids, skip_special_tokens=True)

        print("\n--- Generated Response ---")
        print(response_text)
        print("--------------------------")

        # 次の対話のために更新された会話履歴を返す (オプション)
        # updated_messages = current_messages + [{"role": "assistant", "content": response_text.strip()}]
        # return updated_messages


    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    # --- 前回の対話のセットアップ (仮定) ---
    initial_question = "What do you call a sweet eaten after dinner?"
    # 仮のモデル応答。実際には、この応答は最初の質問に対してモデルが生成したものです。
    # ここでは、モデルが "Dessert." と応答したとします。
    # より自然な応答として "That would be dessert." や "It's called a dessert." も考えられます。
    # モデルの挙動によってこの部分は変わります。
    assumed_initial_assistant_response = "A post-dinner treat or dessert."

    # (オプション) システムプロンプト
    system_prompt = {"role": "system", "content": "You are a helpful and knowledgeable assistant that correctly identifies words and can manipulate them as requested."}

    # これまでの会話履歴を構築
    # システムプロンプトは最初に入れるのが一般的です
    conversation_history = [
        system_prompt,
        {"role": "user", "content": initial_question},
        {"role": "assistant", "content": assumed_initial_assistant_response}
    ]

    # --- 今回の追加の問いかけ ---
    follow_up_user_question = "Please give me the plural form of the word with its spelling in reverse order."

    # 追加の問いかけに対する応答を生成
    generate_follow_up_chat_response(conversation_history, follow_up_user_question)

Loading model and tokenizer for 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' (if not already cached)...
Model is on device: cuda:0

--- Generated Prompt for the Follow-up Question (including history) ---
<|system|>
You are a helpful and knowledgeable assistant that correctly identifies words and can manipulate them as requested.</s>
<|user|>
What do you call a sweet eaten after dinner?</s>
<|assistant|>
A post-dinner treat or dessert.</s>
<|user|>
Please give me the plural form of the word with its spelling in reverse order.</s>
<|assistant|>

---------------------------------------------------------------------

Generating response to the follow-up question...

--- Generated Response ---
The plural form of the word with its spelling in reverse order is "shirts" and "shirts" are the plural form.
--------------------------


In [ ]:
# (generate_follow_up_chat_response 関数の定義は前回と同じとします)

if __name__ == "__main__":
    initial_question = "What do you call a sweet eaten after dinner?"
    # モデルが実際に生成した応答を使用
    actual_initial_assistant_response = "A post-dinner treat or dessert."

    # ユーザーが操作対象としたい単語を明確にする (ここでは 'dessert' を選択)
    # 実際の応用では、この選択をもっと動的に行うか、ユーザーに確認するステップが必要かもしれません。
    # 今回は、アシスタントの応答に含まれる単語から一つ選びます。
    # 簡単な抽出ロジック（例：最後の名詞）や、より高度なNLUが必要になることもあります。
    # ここでは手動で "dessert" を指定します。
    if "dessert" in actual_initial_assistant_response.lower():
        target_word_for_manipulation = "dessert"
    elif "treat" in actual_initial_assistant_response.lower():
        target_word_for_manipulation = "treat"
    else:
        # 適切な単語が見つからない場合のフォールバック
        print("Warning: Could not determine a specific target word from the assistant's previous response for manipulation.")
        target_word_for_manipulation = "word" # 曖昧なままにするか、エラー処理

    # 修正されたフォローアップ質問
    follow_up_user_question_explicit = (
        f"Regarding the word '{target_word_for_manipulation}' you mentioned, "
        f"please give me its plural form, and then provide the spelling of that plural form in reverse order."
    )
    # よりステップを分けた指示も有効かもしれません:
    # follow_up_user_question_explicit = (
    # f"Let's focus on the word '{target_word_for_manipulation}'. "
    # f"First, what is its plural form? "
    # f"Second, what is the spelling of that plural form in reverse order?"
    # )


    system_prompt = {"role": "system", "content": "You are a helpful and knowledgeable assistant that correctly identifies words and can manipulate them as requested."}

    conversation_history = [
        system_prompt,
        {"role": "user", "content": initial_question},
        {"role": "assistant", "content": actual_initial_assistant_response},
        # ここで修正された質問を使う
        # {"role": "user", "content": follow_up_user_question_explicit} # この行は generate_follow_up_chat_response に渡す
    ]

    # 追加の問いかけに対する応答を生成
    # generate_follow_up_chat_response 関数の呼び出し時に、
    # 2番目の引数として follow_up_user_question_explicit を渡します。
    print(f"\n--- Using modified follow-up question for clarity ---")
    print(f"Modified question: {follow_up_user_question_explicit}")
    generate_follow_up_chat_response(conversation_history, follow_up_user_question_explicit) # 修正後の質問を使用


--- Using modified follow-up question for clarity ---
Modified question: Regarding the word 'dessert' you mentioned, please give me its plural form, and then provide the spelling of that plural form in reverse order.
Loading model and tokenizer for 'TinyLlama/TinyLlama-1.1B-Chat-v1.0' (if not already cached)...
Model is on device: cuda:0

--- Generated Prompt for the Follow-up Question (including history) ---
<|system|>
You are a helpful and knowledgeable assistant that correctly identifies words and can manipulate them as requested.</s>
<|user|>
What do you call a sweet eaten after dinner?</s>
<|assistant|>
A post-dinner treat or dessert.</s>
<|user|>
Regarding the word 'dessert' you mentioned, please give me its plural form, and then provide the spelling of that plural form in reverse order.</s>
<|assistant|>

---------------------------------------------------------------------

Generating response to the follow-up question...

--- Generated Response ---
Sure, the plural form of th

## 96. プロンプトによる感情分析

事前学習済み言語モデルで感情分析を行いたい。テキストを含むプロンプトを事前学習済み言語モデルに与え、（ファインチューニングは行わずに）テキストのポジネガを予測するという戦略で、[SST-2](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip)の開発データにおける正解率を測定せよ。

In [1]:
%%capture
!pip install scikit-learn
!pip install numpy
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
!unzip SST-2.zip

In [3]:
import pandas as pd

# データの読み込み（タブ区切り、ヘッダーあり）
train_df = pd.read_csv("SST-2/train.tsv", sep='\t', header=0)
dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t', header=0)

In [17]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from sklearn.metrics import accuracy_score
from tqdm import tqdm # 進捗表示のため

# 事前準備：dev_df がロード済みであること
# import pandas as pd
# dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t', header=0)
# 上記はユーザーが既に実行していると仮定します。

# 0. デバイス設定 (GPUが利用可能であればGPUを使用)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1. モデルとトークナイザのロード
model_name = "gpt2-large" # "gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl" などが利用可能
try:
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")
    print("Please ensure you have an internet connection, the 'transformers' library is installed,")
    print("and the model name is correct.")
    exit() # エラーが発生した場合は終了

model.to(device)
model.eval() # 推論モードに設定

# GPT-2 tokenizer はデフォルトで pad_token を持たないため、eos_token で代用
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

# 2. 感情分析関数 (ゼロショット)
def predict_sentiment_gpt2_zero_shot(text, model, tokenizer, device):
    # プロンプトはユーザーが変更したものを採用
    # 例1: 単一単語での回答を強く促す
    #prompt = f"Is the sentiment of the following review 'positive' or 'negative'? Answer with only one word: 'positive' or 'negative'.\nReview: \"{text}\"\nSentiment:"
    # 例2: Few-Shot プロンプト
    prompt = f"""Determine if the sentiment of the following movie reviews is 'positive' or 'negative'.

    Review: "This movie was absolutely fantastic! The acting was superb and the plot was gripping."
    Sentiment: positive

    Review: "I was really disappointed with this film. It was boring and predictable."
    Sentiment: negative

    Review: "A truly moving and uplifting experience. I highly recommend it."
    Sentiment: positive

    Review: "The plot made no sense and the characters were unlikeable. A waste of time."
    Sentiment: negative

    Review: "{text}"
    Sentiment:""" # モデルにこの後に "positive" または "negative" を生成させる

    inputs = tokenizer(prompt, return_tensors="pt", max_length=(1024 - 10), truncation=True, padding=False)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=2,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False
        )

    prompt_length = input_ids.shape[1]
    generated_text_ids = outputs[0][prompt_length:]
    prediction_text = tokenizer.decode(generated_text_ids, skip_special_tokens=True).strip().lower()

    # 予測の解釈を修正
    is_positive = "positive" in prediction_text
    is_negative = "negative" in prediction_text

    # デバッグ用に生成されたテキストを確認したい場合は以下のコメントを解除
    if not is_positive and not is_negative:
        print(f"Text: \"{text[:30]}...\", Generated for undecided: \"{prediction_text}\"")
    elif is_positive and is_negative:
         print(f"Text: \"{text[:30]}...\", Generated for ambiguous: \"{prediction_text}\"")


    if is_positive and not is_negative:  # "positive" のみ明確に含まれる場合
        return 1
    elif is_negative and not is_positive:  # "negative" のみ明確に含まれる場合
        return 0
    # elif is_positive and is_negative:  # "positive" と "negative" が両方含まれる稀なケース
    #     return -1 # 分類不能（アンビギュアス）として扱う
    else:  # "positive" も "negative" も含まれない、または両方含まれる場合
        return -1 # 分類不能（不明）として扱う

# 3. SST-2開発データでの評価
predictions = []
true_labels = []

print("Starting sentiment prediction on SST-2 dev set...")
# dev_df がロード済みであることを確認 (ユーザー提供のコードでロードされているはず)
if 'dev_df' not in globals():
    print("Error: dev_df is not loaded. Please load your SST-2 dev data first.")
    print("Example: dev_df = pd.read_csv('SST-2/dev.tsv', sep='\\t', header=0)")
    exit()

for index, row in tqdm(dev_df.iterrows(), total=dev_df.shape[0], desc="Processing SST-2 dev"):
    sentence = row['sentence']
    true_label = row['label'] # SST-2のラベルは 0 (negative), 1 (positive)

    predicted_label = predict_sentiment_gpt2_zero_shot(sentence, model, tokenizer, device)

    predictions.append(predicted_label)
    true_labels.append(true_label)

# 4. 正解率の計算と表示
if len(predictions) > 0:
    num_total_samples = len(predictions)

    valid_predictions = []      # 明確な予測 (0 or 1)
    valid_true_labels = []    # 明確な予測に対応する真のラベル
    num_undecided = 0           # 分類不能 (-1) となった予測の数

    for pred, true_val in zip(predictions, true_labels):
        if pred != -1:  # 0 (negative) または 1 (positive) の明確な予測の場合
            valid_predictions.append(pred)
            valid_true_labels.append(true_val)
        else:
            num_undecided += 1

    print(f"\n--- Evaluation Results (with undecided category) ---")
    print(f"Total samples processed: {num_total_samples}")
    print(f"Number of undecided predictions (-1): {num_undecided} ({(num_undecided/num_total_samples)*100:.2f}%)")

    if len(valid_predictions) > 0:
        accuracy_on_decided = accuracy_score(valid_true_labels, valid_predictions)
        print(f"Accuracy on clearly decided samples ({len(valid_predictions)} samples): {accuracy_on_decided:.4f}")

        print("\nPrediction distribution for decided samples (0: negative, 1: positive):")
        pred_counts_decided = pd.Series(valid_predictions).value_counts().sort_index()
        print(pred_counts_decided)

        print("\nTrue label distribution for corresponding decided samples (0: negative, 1: positive):")
        true_counts_for_decided = pd.Series(valid_true_labels).value_counts().sort_index()
        print(true_counts_for_decided)
    else:
        print("No samples were clearly decided by the model (all predictions were undecided).")

    # 全体を通しての予測の分布 (不明なものも含む)
    print("\nOverall prediction distribution (0: negative, 1: positive, -1: undecided):")
    overall_pred_counts = pd.Series(predictions).value_counts().sort_index()
    print(overall_pred_counts)

    print("\nOverall true label distribution (remains the same):")
    true_counts_overall = pd.Series(true_labels).value_counts().sort_index()
    print(true_counts_overall)

else:
    print("No predictions were made. Please check your data and code.")

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Starting sentiment prediction on SST-2 dev set...


Processing SST-2 dev: 100%|██████████| 872/872 [01:46<00:00,  8.22it/s]


--- Evaluation Results (with undecided category) ---
Total samples processed: 872
Number of undecided predictions (-1): 0 (0.00%)
Accuracy on clearly decided samples (872 samples): 0.8979

Prediction distribution for decided samples (0: negative, 1: positive):
0    447
1    425
Name: count, dtype: int64

True label distribution for corresponding decided samples (0: negative, 1: positive):
0    428
1    444
Name: count, dtype: int64

Overall prediction distribution (0: negative, 1: positive, -1: undecided):
0    447
1    425
Name: count, dtype: int64

Overall true label distribution (remains the same):
0    428
1    444
Name: count, dtype: int64


In [19]:
from sklearn.metrics import accuracy_score, classification_report # classification_report を追加

# 4. 正解率の計算と表示 (修正版 - さらに評価指標を追加)
if len(predictions) > 0: # predictions リストにはモデルの予測結果 (0, 1, -1) が入っている
    num_total_samples = len(predictions)

    valid_predictions = []      # 明確な予測 (0 or 1)
    valid_true_labels = []    # 明確な予測に対応する真のラベル
    num_undecided = 0           # 分類不能 (-1) となった予測の数

    for pred, true_val in zip(predictions, true_labels):
        if pred != -1:  # 0 (negative) または 1 (positive) の明確な予測の場合
            valid_predictions.append(pred)
            valid_true_labels.append(true_val)
        else:
            num_undecided += 1

    print(f"\n--- Evaluation Results ---")
    print(f"Model: gpt2-large (with Few-Shot Prompt)") # モデル情報を追記すると分かりやすい
    print(f"Total samples processed: {num_total_samples}")

    if num_undecided > 0: # 今回の結果ではここは通りませんが、汎用性のために残します
        print(f"Number of undecided predictions (-1): {num_undecided} ({(num_undecided/num_total_samples)*100:.2f}%)")

    if len(valid_predictions) > 0:
        # 精度 (Accuracy)
        accuracy_on_decided = accuracy_score(valid_true_labels, valid_predictions)
        print(f"Accuracy on decided samples ({len(valid_predictions)} samples): {accuracy_on_decided:.4f}")

        # 適合率、再現率、F1スコアなどをまとめて表示
        # target_names でラベルの意味を指定するとレポートが読みやすくなります
        report = classification_report(
            valid_true_labels,
            valid_predictions,
            target_names=['negative (0)', 'positive (1)']
        )
        print("\nClassification Report:")
        print(report)

        # 参考: 予測分布と実際のラベル分布 (変更なし)
        print("\nPrediction distribution for decided samples (0: negative, 1: positive):")
        pred_counts_decided = pd.Series(valid_predictions).value_counts().sort_index()
        print(pred_counts_decided)

        print("\nTrue label distribution for corresponding decided samples (0: negative, 1: positive):")
        true_counts_for_decided = pd.Series(valid_true_labels).value_counts().sort_index()
        print(true_counts_for_decided)
    else:
        # この分岐は num_undecided == num_total_samples の場合に相当
        print("No samples were clearly decided by the model (all predictions were undecided).")

else:
    print("No predictions were made. Please check your data and code.")



--- Evaluation Results ---
Model: gpt2-large (with Few-Shot Prompt)
Total samples processed: 872
Accuracy on decided samples (872 samples): 0.8979

Classification Report:
              precision    recall  f1-score   support

negative (0)       0.88      0.92      0.90       428
positive (1)       0.92      0.88      0.90       444

    accuracy                           0.90       872
   macro avg       0.90      0.90      0.90       872
weighted avg       0.90      0.90      0.90       872


Prediction distribution for decided samples (0: negative, 1: positive):
0    447
1    425
Name: count, dtype: int64

True label distribution for corresponding decided samples (0: negative, 1: positive):
0    428
1    444
Name: count, dtype: int64


In [ ]:
import pandas as pd

# SST-2データセットのパスを適宜修正してください
try:
    dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t', header=0)
except FileNotFoundError:
    print("SST-2/dev.tsv が見つかりません。ファイルパスを確認してください。")
    # ここでは処理を中断するか、サンプルデータで続行するかを選択できます。
    # 例として、サンプルデータフレームを作成します（実際のSST-2データではありません）
    print("代わりにサンプルデータを使用します。")
    data = {'sentence': ["this is a wonderful movie", "this is a terrible film", "it was an okay experience"],
            'label': [1, 0, 1]} # SST-2ではニュートラルは通常ありませんが、例として含めます。
                                # SST-2の実際のラベルは0 (negative) と 1 (positive) です。
    dev_df = pd.DataFrame(data)

# データの中身を確認 (最初の数行)
print("SST-2 開発データ (dev_df):")
print(dev_df.head())
print(f"データ数: {len(dev_df)}")

SST-2 開発データ (dev_df):
                                            sentence  label
0    it 's a charming and often affecting journey .       1
1                 unflinchingly bleak and desperate       0
2  allows us to hope that nolan is poised to emba...      1
3  the acting , costumes , music , cinematography...      1
4                  it 's slow -- very , very slow .       0
データ数: 872


In [ ]:
from transformers import pipeline
from tqdm import tqdm # 進捗表示用

# ゼロショット分類パイプラインの準備
# device=0を指定するとGPUを使用します。GPUがない場合は-1（CPU）を指定します。
try:
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0 if torch.cuda.is_available() else -1)
    print(f"Using model: facebook/bart-large-mnli on {'GPU' if torch.cuda.is_available() else 'CPU'}")
except Exception as e:
    print(f"モデルのロード中にエラーが発生しました: {e}")
    print("CPUでデフォルトモデルを使用して続行します。")
    classifier = pipeline("zero-shot-classification", device=-1) # より汎用的なデフォルトモデル

# 感情ラベルの定義 (SST-2のラベル 0: negative, 1: positive に対応させる)
candidate_labels = ["negative", "positive"]
label_map = {"negative": 0, "positive": 1}

predictions = []
predicted_labels_text = []

# dev_dfの各文に対して予測を実行
# 大量のデータがある場合、時間がかかることがあります。
# APIのレート制限やメモリに注意し、必要であればバッチ処理を検討してください。
# ここでは簡単のため一件ずつ処理します。
print(f"\n{len(dev_df)}件のテキストで予測を開始します...")
for sentence in tqdm(dev_df['sentence']):
    if not sentence or not isinstance(sentence, str): # 空の文や非文字列データをスキップ
        predictions.append(-1) # エラーまたはスキップを示す値
        predicted_labels_text.append("N/A")
        continue
    try:
        # パイプラインで予測
        result = classifier(sentence, candidate_labels, multi_label=False) # multi_label=False で最もスコアの高いラベルのみ取得
        predicted_label_text = result['labels'][0]
        predicted_labels_text.append(predicted_label_text)
        predictions.append(label_map[predicted_label_text])
    except Exception as e:
        print(f"エラー発生: 文「{sentence}」, エラー: {e}")
        predictions.append(-1) # エラーを示す値（後で処理から除外するため）
        predicted_labels_text.append("Error")


dev_df['predicted_label_text'] = predicted_labels_text
dev_df['predicted_label'] = predictions

# 予測結果の確認 (最初の数行)
print("\n予測結果:")
print(dev_df[['sentence', 'label', 'predicted_label_text', 'predicted_label']].head())

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


モデルのロード中にエラーが発生しました: name 'torch' is not defined
CPUでデフォルトモデルを使用して続行します。


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu



872件のテキストで予測を開始します...


100%|██████████| 872/872 [18:28<00:00,  1.27s/it]


予測結果:
                                            sentence  label  \
0    it 's a charming and often affecting journey .       1   
1                 unflinchingly bleak and desperate       0   
2  allows us to hope that nolan is poised to emba...      1   
3  the acting , costumes , music , cinematography...      1   
4                  it 's slow -- very , very slow .       0   

  predicted_label_text  predicted_label  
0             positive                1  
1             negative                0  
2             positive                1  
3             positive                1  
4             negative                0  


## 97. 埋め込みに基づく感情分析

事前学習済み言語モデルでテキストをベクトルで表現（エンコード）し、そのベクトルにフィードフォワード層を通すことで極性ラベルを予測するモデルを学習せよ。

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
# from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup # <--- 変更点 (削除)
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup # <--- 変更点
from torch.optim import AdamW # <--- 変更点: torch.optimからAdamWをインポート
from sklearn.metrics import accuracy_score
import torch.nn as nn
from tqdm import tqdm

# 0. 設定値
MODEL_NAME = 'distilbert-base-uncased' # 事前学習済みモデル
MAX_LEN = 128  # トークン化する際の最大長
BATCH_SIZE = 16
EPOCHS = 1
LEARNING_RATE = 2e-5
NUM_CLASSES = 2 # ポジティブ(1), ネガティブ(0) の2クラス

# GPUが利用可能か確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 1. データの読み込み
try:
    train_df = pd.read_csv("SST-2/train.tsv", sep='\t', header=0)
    dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t', header=0)
    print(f"Train data loaded: {len(train_df)} samples")
    print(f"Dev data loaded: {len(dev_df)} samples")
except FileNotFoundError:
    print("SST-2/train.tsv or SST-2/dev.tsvが見つかりません。ファイルパスを確認してください。")
    print("ダミーデータで続行します。")
    train_data = {'sentence': ["this is a wonderful movie", "this is a terrible film", "another great one", "very bad", "loved it"] * 10,
                  'label': [1, 0, 1, 0, 1] * 10}
    dev_data = {'sentence': ["pretty good", "not my cup of tea", "fantastic"],
                'label': [1, 0, 1]}
    train_df = pd.DataFrame(train_data)
    dev_df = pd.DataFrame(dev_data)

# 2. トークナイザの準備
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 3. PyTorch Datasetの定義
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 4. DataLoaderの作成関数
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = SentimentDataset(
        texts=df.sentence.to_numpy(),
        labels=df.label.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=2 # 環境に応じて調整
    )

train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
dev_data_loader = create_data_loader(dev_df, tokenizer, MAX_LEN, BATCH_SIZE)

# 5. モデルの定義
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes, model_name):
        super(SentimentClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        hidden_size = self.bert.config.hidden_size
        self.dropout = nn.Dropout(p=0.3)
        self.ffn = nn.Linear(hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = outputs.last_hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.ffn(pooled_output)
        return logits

model = SentimentClassifier(NUM_CLASSES, MODEL_NAME)
model = model.to(device)

# 6. オプティマイザと損失関数の設定
# optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, correct_bias=False) # <--- 変更点 (削除)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE) # <--- 変更点: correct_biasパラメータを削除
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

# 7. 学習関数
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for batch_idx, d in tqdm(enumerate(data_loader), total=len(data_loader), desc="Training"):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, labels)

        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, sum(losses) / len(losses)

# 8. 評価関数
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for d in tqdm(data_loader, total=len(data_loader), desc="Evaluating"):
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, labels)

            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    avg_loss = sum(losses) / len(losses)
    return accuracy, avg_loss

# 9. 学習ループの実行
history = {'train_acc': [], 'train_loss': [], 'val_acc': [], 'val_loss': []}

best_accuracy = 0
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(train_df)
    )
    print(f'Train loss {train_loss:.4f} accuracy {train_acc:.4f}')

    val_acc, val_loss = eval_model(
        model,
        dev_data_loader,
        loss_fn,
        device,
        len(dev_df)
    )
    print(f'Val   loss {val_loss:.4f} accuracy {val_acc:.4f}')
    print()

    history['train_acc'].append(train_acc.item() if torch.is_tensor(train_acc) else train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc.item() if torch.is_tensor(val_acc) else val_acc)
    history['val_loss'].append(val_loss)

    if val_acc > best_accuracy:
        best_accuracy = val_acc

print(f"学習完了。開発データでの最高正解率: {best_accuracy:.4f}")

Using device: cuda
Train data loaded: 67349 samples
Dev data loaded: 872 samples
Epoch 1/1
----------


Training: 100%|██████████| 4210/4210 [12:00<00:00,  5.84it/s]


Train loss 0.2259 accuracy 0.9157


Evaluating: 100%|██████████| 55/55 [00:03<00:00, 16.33it/s]

Val   loss 0.3125 accuracy 0.8968

学習完了。開発データでの最高正解率: 0.8968


## 98. ファインチューニング

問題96のプロンプトに対して、正解の感情ラベルをテキストの応答として返すように事前学習済みモデルをファインチューニングせよ。

In [2]:
import pandas as pd
import torch
from datasets import Dataset # DatasetDict はここでは不要かもしれません
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from sklearn.metrics import accuracy_score
import numpy as np

# SST-2データセットのパスを適宜修正してください
try:
    train_df_orig = pd.read_csv("SST-2/train.tsv", sep='\t', header=0)
    dev_df_orig = pd.read_csv("SST-2/dev.tsv", sep='\t', header=0)
    print(f"Original train data loaded: {len(train_df_orig)} samples")
    print(f"Original dev data loaded: {len(dev_df_orig)} samples")
except FileNotFoundError:
    print("SST-2/train.tsv or SST-2/dev.tsv が見つかりません。ファイルパスを確認してください。")
    print("代わりにサンプルデータを使用します。")
    train_data_dict = {'sentence': ["this is a wonderful movie", "this is a terrible film", "another great one for the books", "not good at all", "i enjoyed it immensely"], 'label': [1, 0, 1, 0, 1]}
    dev_data_dict = {'sentence': ["pretty good experience", "i hated every moment", "it was okay, not great"], 'label': [1, 0, 0]}
    train_df_orig = pd.DataFrame(train_data_dict)
    dev_df_orig = pd.DataFrame(dev_data_dict)

# ラベルをテキストに変換する辞書
label_to_text = {0: "negative", 1: "positive"}

# プロンプト形式の定義
def create_prompted_input(sentence):
    return f"What is the sentiment of this sentence? Sentence: {sentence}"

# モデル名とトークナイザの指定
MODEL_NAME = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# データフレームの準備 (サンプル数を減らす場合はここで調整)
train_df = train_df_orig
dev_df = dev_df_orig

# Hugging Face Datasets形式に変換
train_dataset_hf = Dataset.from_pandas(train_df)
dev_dataset_hf = Dataset.from_pandas(dev_df)

# <<<ここからが修正されたデータ前処理部分>>>

# バッチ処理用の前処理関数
def preprocess_function_batched(examples): # 'examples' は列名をキーとするリストの辞書
    inputs = [create_prompted_input(sentence) for sentence in examples['sentence']]
    targets = [label_to_text[label] for label in examples['label']]

    # 入力テキストのトークナイズ
    model_inputs = tokenizer(inputs, max_length=128, padding="max_length", truncation=True)

    # ターゲットテキストのトークナイズ (T5の作法)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=16, padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# .map() を使用してデータセット全体をトークナイズ (batched=True を使用)
tokenized_train_dataset = train_dataset_hf.map(
    preprocess_function_batched,
    batched=True,
    remove_columns=train_df.columns.tolist() # 元の'sentence', 'label'列を削除
)
tokenized_dev_dataset = dev_dataset_hf.map(
    preprocess_function_batched,
    batched=True,
    remove_columns=dev_df.columns.tolist() # 元の'sentence', 'label'列を削除
)
# <<<修正されたデータ前処理部分ここまで>>>

print("\nTokenized train dataset example:")
if len(tokenized_train_dataset) > 0:
    print(tokenized_train_dataset[0])
else:
    print("Tokenized train dataset is empty.")

# ... (以降のモデルロード、トレーニング引数、Trainerの初期化、学習実行のコードは前回と同様) ...
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)

# データコレータ: バッチ内のシーケンスをパディングします
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# トレーニング引数
training_args = Seq2SeqTrainingArguments(
    output_dir="./results_sst2_t5_finetuned",
    num_train_epochs=1,  # デモ用に1エポック。実際は3-5程度
    per_device_train_batch_size=8,  # GPUメモリに応じて調整
    per_device_eval_batch_size=8,   # GPUメモリに応じて調整
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_sst2_t5',
    logging_steps=100,
    eval_strategy="epoch",    # エポックごとに評価
    save_strategy="epoch",          # エポックごとに保存
    load_best_model_at_end=True,    # 最後に最良モデルをロード
    predict_with_generate=True,     # 生成タスクなのでTrue
    fp16=torch.cuda.is_available(), # GPUが利用可能なら半精度浮動小数点数で高速化
    # report_to="tensorboard" # TensorBoardでログを見たい場合
    report_to="none",  # <--- この行を追加して wandb を無効化
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # predictions は generate された token id の配列
    # labels も token id の配列 (ただし、パディングトークンは -100 になっている)

    # デコードしてテキストに戻す
    # pad_token_id をスキップし、特殊トークンもスキップ
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # labels の -100 を pad_token_id に置き換えてデコード
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 簡単な後処理 (例: 余分なスペースの除去)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # 正解率の計算
    correct = 0
    for pred, label in zip(decoded_preds, decoded_labels):
        if pred == label:
            correct += 1

    accuracy = correct / len(decoded_labels)
    return {"accuracy": accuracy}

# Trainerの初期化
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_dev_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics, # 評価指標関数を指定
)

# 学習の開始
print("\nファインチューニングを開始します...")
trainer.train()

# 学習結果の評価 (最終的な最良モデルで)
print("\n最終モデルでの評価:")
eval_results = trainer.evaluate()
print(eval_results)

# ファインチューニングされたモデルで推論
def predict_sentiment(sentence_text, trained_model, tokenizer_loaded, device_name):
    prompted_text = create_prompted_input(sentence_text)
    inputs = tokenizer_loaded(prompted_text, return_tensors="pt", max_length=128, truncation=True, padding=True).to(device_name)

    trained_model.eval() # 評価モード
    with torch.no_grad():
        outputs = trained_model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=10, # "positive" or "negative" なので短くて良い
            num_beams=5, # ビームサーチのビーム数
            early_stopping=True
        )

    predicted_text = tokenizer_loaded.decode(outputs[0], skip_special_tokens=True)
    return predicted_text.strip()

# 推論例
print("\nファインチューニングされたモデルでの推論例:")
test_sentences = [
    "This movie was absolutely fantastic!",
    "I would not recommend this film to anyone.",
    "It was an okay movie, nothing special.",
    "The acting was superb and the plot was engaging."
]

# モデルとトークナイザをロード (Trainerが最後にベストモデルをロードしているはず)
# もし明示的に保存・ロードする場合は以下のようにします
# model_path = "./results_sst2_t5_finetuned/checkpoint-XXXX" # 最良のチェックポイント
# loaded_model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
# loaded_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME) # or model_path if saved there

loaded_model = trainer.model # Trainerが保持しているモデル (load_best_model_at_end=True の場合)

for sentence in test_sentences:
    sentiment = predict_sentiment(sentence, loaded_model, tokenizer, device)
    print(f"Sentence: {sentence}\nPredicted Sentiment: {sentiment}\n")

# モデルを保存したい場合
# trainer.save_model("./my_finetuned_t5_sst2_model")
# tokenizer.save_pretrained("./my_finetuned_t5_sst2_model")

KeyboardInterrupt: 

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 36.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [17]:
import pandas as pd
import torch
from torch.utils.data import Dataset # DataLoader は Trainer が内部で使うので直接は不要かも
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from sklearn.metrics import accuracy_score, classification_report
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

# --- 1. データ読み込みとプロンプト定義 (前回と同様) ---
# BASE_PROMPT_EXAMPLES, PROMPT_INPUT_SUFFIX, train_df, dev_df の準備
# Few-Shot Prompt (固定部分)
BASE_PROMPT_EXAMPLES = """Determine if the sentiment of the following movie reviews is 'positive' or 'negative'.

Review: "This movie was absolutely fantastic! The acting was superb and the plot was gripping."
Sentiment: positive

Review: "I was really disappointed with this film. It was boring and predictable."
Sentiment: negative

Review: "A truly moving and uplifting experience. I highly recommend it."
Sentiment: positive

Review: "The plot made no sense and the characters were unlikeable. A waste of time."
Sentiment: negative

Review: """ # ここに実際のレビュー文が続く

PROMPT_INPUT_SUFFIX = "\nSentiment:" # この後にモデルが " positive" または " negative" を生成する

# SST-2データの読み込み (ファイルパスは適宜調整してください)
try:
    train_df_full = pd.read_csv("SST-2/train.tsv", sep='\t')
    dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t')
except FileNotFoundError:
    print("Error: SST-2 data files not found. Please check the paths.")
    exit()

# ラベルを文字列に変換
def label_to_text(label):
    return " positive" if label == 1 else " negative" # 先頭にスペース

train_df_full['text_label'] = train_df_full['label'].apply(label_to_text)
dev_df['text_label'] = dev_df['label'].apply(label_to_text)

# (オプション) train_df_full を学習用と小規模な検証用に分割する場合
# train_df, val_df = train_test_split(train_df_full, test_size=0.1, random_state=42)
# 今回は train_df_full を学習に、dev_df を最終評価に使う想定
train_df = train_df_full

# (SentimentDataset クラスの定義も前回と同様。ただし、paddingの修正は適用済みとします)
class SentimentDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, tokenizer: GPT2Tokenizer, max_length: int = 300):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.texts = dataframe['sentence'].tolist()
        self.labels_text = dataframe['text_label'].tolist() # 例: " positive", " negative"

        if self.tokenizer.pad_token is None:
            print("Warning: Tokenizer pad_token is None. Setting to eos_token. "
                  "Ensure this is intended for your DataCollator.")
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def __len__(self) -> int:
        return len(self.texts)

    def __getitem__(self, idx: int) -> dict:
        review_text: str = self.texts[idx]
        target_text: str = self.labels_text[idx] # 例: " positive"

        # プロンプトの入力部分 (Few-Shot例 + 実際のレビュー + "Sentiment:")
        prompt_input_part: str = BASE_PROMPT_EXAMPLES + f'"{review_text}"' + PROMPT_INPUT_SUFFIX

        # モデルへの入力全体 (プロンプト入力 + ターゲットテキスト)
        full_prompt_text: str = prompt_input_part + target_text

        # フルテキストをトークナイズ
        inputs = self.tokenizer(
            full_prompt_text,
            truncation=True,                # max_length を超えたら切り詰め
            max_length=self.max_length,
            padding=False,                  # パディングはDataCollatorで行うのでここではしない
            return_tensors="pt"             # PyTorchテンソルで返す
        )

        input_ids = inputs.input_ids.squeeze(0)         # バッチ次元を削除 (単一サンプルのため)
        attention_mask = inputs.attention_mask.squeeze(0) # 同上

        # ラベルを作成 (input_idsをコピー)
        labels = input_ids.clone()

        # プロンプト部分の長さを計算して、labelsのその部分を-100でマスクする
        # prompt_input_part をトークナイズして、そのトークン数をsource_lenとする
        # この際、full_prompt_textのトークナイズ方法と一貫性を持たせることが重要
        # (特に special_tokens の扱いや truncation の影響)
        tokenized_prompt_input_part = self.tokenizer(
            prompt_input_part,
            max_length=self.max_length,   # full_prompt_text と同じ max_length を適用
            truncation=True,              # prompt_input_part 自体が長い場合も切り詰める
            add_special_tokens=True       # userの前回コードに合わせたが、Falseの方が安全な場合も。
                                          # GPT2Tokenizerは通常文字列にBOS/EOSを勝手に追加しない。
                                          # Trueにするとtokenizerの設定次第でBOS/EOSが付く可能性がある。
                                          # Falseが無難かもしれないが、userのコードに合わせる。
        )
        source_len = len(tokenized_prompt_input_part.input_ids)

        actual_input_len = input_ids.shape[0] # 実際にトークナイズされた全体の長さ

        # labels のマスク処理
        if source_len < actual_input_len:
            # prompt_input_part の方が全体より短い場合 (つまりターゲットテキストが含まれている)
            # prompt_input_part に対応する部分をマスク
            labels[:source_len] = -100
        else:
            # prompt_input_part の長さが全体と同じかそれ以上の場合
            # (ターゲットテキストが完全に切り詰められたか、空だった場合)
            # 予測すべきターゲットがないので、全てのラベルをマスク
            labels[:] = -100

        # デバッグ用のprint文 (通常はコメントアウト)
        print(f"Sample idx {idx}: source_len={source_len}, actual_input_len={actual_input_len}")
        print(f"Sample idx {idx}: input_ids.shape={input_ids.shape}, attention_mask.shape={attention_mask.shape}, labels.shape={labels.shape}")
        if idx < 1: # 最初のサンプルだけ中身を確認
            print("--- Sample Debug ---")
            print("Review Text:", review_text)
            print("Target Text:", target_text)
            print("Prompt Input Part (first 50 chars):", prompt_input_part[:50])
            print("Full Prompt Text (first 50 chars):", full_prompt_text[:50])
            print("Decoded input_ids:", self.tokenizer.decode(input_ids))
            print("Labels (masked):", labels.tolist())
            print("Decoded labels (where not -100):", self.tokenizer.decode([l if l != -100 else self.tokenizer.pad_token_id for l in labels]))
            print("--- End Sample Debug ---")

        # 最終的な形状チェック (バッチ処理エラーを防ぐため)
        assert input_ids.shape[0] == attention_mask.shape[0], \
            f"Idx {idx}: input_ids length ({input_ids.shape[0]}) != attention_mask length ({attention_mask.shape[0]})"
        assert input_ids.shape[0] == labels.shape[0], \
            f"Idx {idx}: input_ids length ({input_ids.shape[0]}) != labels length ({labels.shape[0]})"

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

# --- 2. モデルとトークナイザの初期化 (LoRAを適用) ---
model_name = "gpt2-large"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# パディングトークンの設定
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    # model.config.pad_token_id はPEFTモデル取得後に設定しても良いが、ここで設定しておいても問題ない

# ベースモデルのロード
# (オプション) もしさらにメモリを削減したい場合、ここでbitsandbytesで量子化してロードすることも可能
# model = GPT2LMHeadModel.from_pretrained(model_name, load_in_8bit=True) # 8bit量子化
# model = prepare_model_for_kbit_training(model) # 8bit/4bit量子化の準備

# model = GPT2LMHeadModel.from_pretrained(model_name)
# if tokenizer.pad_token is None: # 再度確認 (モデルロード後に tokenizer の設定が影響受ける場合があるため)
#     tokenizer.pad_token = tokenizer.eos_token

# 4ビット量子化の設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",          # 量子化タイプ (nf4が推奨されることが多い)
    bnb_4bit_compute_dtype=torch.float16, # 計算時のデータ型 (bfloat16も可)
    bnb_4bit_use_double_quant=True,     # 二重量子化でさらに効率化
)

model = GPT2LMHeadModel.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.pad_token_id = tokenizer.eos_token_id

# 8ビット/4ビット量子化モデルの学習準備 (特にgradient_checkpointingと併用する場合に推奨)
# training_args.gradient_checkpointing が True の場合を想定
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True) # training_args.gradient_checkpointing を渡す

# LoRAの設定
lora_config = LoraConfig(
    r=8,  # LoRAのランク (4, 8, 16, 32などが一般的。小さいほどパラメータ数が少なく、メモリ効率が良い)
    lora_alpha=16, # LoRAのスケーリング係数 (rの2倍などが一般的)
    target_modules=["c_attn"], # GPT-2の場合、'c_attn'がQKV射影に使われる主要な線形層です。
                               # 他の候補: "c_proj" (attention出力射影), "c_fc" (FFNの一部)
                               # model.named_modules() でモジュール名を確認して調整可能
    lora_dropout=0.05,         # LoRAレイヤーのドロップアウト率
    bias="none",               # バイアス項の扱い ("none", "all", "lora_only")
                               # "none" はバイアスを学習せず、メモリ効率が良い
    task_type=TaskType.CAUSAL_LM # GPT-2はCausal LM (自己回帰言語モデル)
)

# PEFTモデルの作成
model = get_peft_model(model, lora_config)

# 学習可能なパラメータの数と割合を表示 (大幅に削減されているはず)
model.print_trainable_parameters()

# GPUが利用可能ならGPUへ
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # PEFTモデルをGPUへ

# --- 3. データセットとデータコレータの準備 (前回と同様) ---
train_dataset = SentimentDataset(train_df, tokenizer, max_length=300) # max_lengthは要調整
dev_dataset = SentimentDataset(dev_df, tokenizer, max_length=300)   # max_lengthは要調整
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False, # これが重要。Causal LM (GPT-2のような) の場合は False
)
# SentimentDatasetの __getitem__ 内の padding="max_length" は削除し、
# DataCollator にパディングを任せるように修正済みであることを前提とします。

# --- 4. TrainingArguments と Trainer (調整の可能性あり) ---
output_dir = "./sst2_gpt2_large_lora_finetuned"
# TrainingArguments は前回の設定をベースに、メモリ状況を見て調整
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,  # LoRAは少ないエポックでも収束しやすい場合がある
    per_device_train_batch_size=2,  # まずは小さく始める (LoRAならもう少し増やせる可能性あり)
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8, # 実質バッチサイズを維持
    learning_rate=1e-4,  # LoRAの場合、少し高めの学習率が良い結果を出すことがある (例: 5e-5, 1e-4, 2e-4)
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
    fp16=True,  # 引き続き有効
    gradient_checkpointing=True, # 引き続き有効 (PEFTモデルでも使えるはず)
    # report_to="tensorboard",
    report_to="none",  # <--- この行を追加して wandb を無効化
)

trainer = Trainer(
    model=model, # LoRA適用済みのモデル
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# --- 5. ファインチューニングの開始と評価 (前回と同様) ---
print("Starting LoRA fine-tuning...")
trainer.train()

# 最良モデル(LoRAアダプタのみ)の保存
trainer.save_model(f"{output_dir}/best_lora_model") # LoRAアダプタの重みが保存される
# トークナイザも必要なら保存 (ベースモデルと共通なので必須ではないが、一貫性のため)
# tokenizer.save_pretrained(f"{output_dir}/best_lora_model")

print("LoRA Fine-tuning finished. Best LoRA adapter saved.")

# --- 評価 ---
# 評価時は、まずベースのgpt2-largeをロードし、その後LoRAアダプタをロードする
from peft import PeftModel
base_model = GPT2LMHeadModel.from_pretrained(model_name)
finetuned_model = PeftModel.from_pretrained(base_model, f"{output_dir}/best_lora_model")
finetuned_model.to(device)
finetuned_model.eval()
# ... (predict_sentiment_finetuned 関数と評価ループは前回と同様だが、モデルとして finetuned_model を使う)
# 評価関数 (前回と同様のものを流用可能)
def predict_sentiment_finetuned(text, model, tokenizer, device, max_new_tokens=5):
    prompt = BASE_PROMPT_EXAMPLES + f'"{text}"' + PROMPT_INPUT_SUFFIX

    inputs = tokenizer(prompt, return_tensors="pt", max_length=280, truncation=True) # max_lengthはプロンプト長に応じて調整
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id, # 明示的に指定
            # temperature=0.7, # 必要に応じてサンプリングパラメータ調整
            # top_k=50,
            # do_sample=True,
            do_sample=False, # 決定的な出力を得るため
            num_beams=1,     # Greedy search
        )

    prompt_length = input_ids.shape[1]
    generated_ids = outputs[0][prompt_length:]
    prediction_text = tokenizer.decode(generated_ids, skip_special_tokens=True).strip().lower()

    # print(f"Input: ...{text[-50:]}\nPrompt suffix: {PROMPT_INPUT_SUFFIX}\nGenerated: '{prediction_text}'") # デバッグ用

    if "positive" in prediction_text:
        return 1
    elif "negative" in prediction_text:
        return 0
    else:
        # ファインチューニングにより、期待する単語を生成しやすくなっているはず
        # print(f"Warning: Undecided. Generated: '{prediction_text}'")
        # ここでのデフォルト処理は要検討 (例: 0、またはエラーとしてカウント)
        # 理想は "positive" か "negative" (またはその一部) のみを生成すること
        # " positive" or " negative" のような完全一致で判定しても良いかもしれない
        if prediction_text.startswith("positive"): return 1
        if prediction_text.startswith("negative"): return 0

        # それでもダメなら、前回同様の処理
        first_word = prediction_text.split(" ")[0]
        if "positive".startswith(first_word): return 1 # "pos" などでもマッチ
        if "negative".startswith(first_word): return 0 # "neg" などでもマッチ
        return -1 # 不明


# 開発データで評価
predictions_ft = []
true_labels_ft = []
undecided_count_ft = 0

print("\nEvaluating fine-tuned model...")
for index, row in tqdm(dev_df.iterrows(), total=dev_df.shape[0], desc="Evaluating FT Model"):
    sentence = row['sentence']
    true_label_num = row['label'] # 0 or 1

    predicted_label_num = predict_sentiment_finetuned(sentence, finetuned_model, tokenizer, device, max_new_tokens=3) # ターゲットは短いので3トークン程度で十分

    if predicted_label_num != -1:
        predictions_ft.append(predicted_label_num)
        true_labels_ft.append(true_label_num)
    else:
        undecided_count_ft +=1
        # 不明な場合、評価には含めないか、あるいは多数派/少数派クラスに割り当てるなどの処理が必要
        # 今回は、明確に判断できたものだけで評価する
        # もし全件評価したいなら、-1 を例えば0に倒すなど
        # predictions_ft.append(0) # 例: 不明はネガティブ扱い
        # true_labels_ft.append(true_label_num)


# 結果表示 (前回と同様のclassification_reportなど)
if len(predictions_ft) > 0:
    accuracy = accuracy_score(true_labels_ft, predictions_ft)
    report = classification_report(true_labels_ft, predictions_ft, target_names=['negative (0)', 'positive (1)'])

    print(f"\n--- Fine-tuned Model Evaluation Results ---")
    print(f"Total decided predictions: {len(predictions_ft)}")
    print(f"Undecided predictions: {undecided_count_ft}")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(report)
else:
    print("No valid predictions made by the fine-tuned model.")


<ipython-input-17-dc8e98d59dff>:242: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 1,474,560 || all params: 775,504,640 || trainable%: 0.1901
Starting LoRA fine-tuning...
Sample idx 6677: source_len=133, actual_input_len=134
Sample idx 6677: input_ids.shape=torch.Size([134]), attention_mask.shape=torch.Size([134]), labels.shape=torch.Size([134])
Sample idx 49712: source_len=134, actual_input_len=135
Sample idx 49712: input_ids.shape=torch.Size([135]), attention_mask.shape=torch.Size([135]), labels.shape=torch.Size([135])


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

## 99. 選好チューニング

問題96のプロンプトに対して、正解の感情ラベルを含むテキストを望ましい応答、間違った感情ラベルを含むテキストを望ましくない応答として、事前学習済み言語モデルを選好チューニング (preference tuning) を実施せよ。選好チューニングのアルゴリズムとしては、近傍方策最適化 (PPO: Proximal Policy Optimization) や直接選好最適化 (DPO: Direct Preference Optimization) などが考えられる。


In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install torch

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
import torch

MODEL_NAME_SFT = "google/flan-t5-small" # 例: FLAN-T5-small
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_SFT)

# SST-2データセットの読み込み (pandasを使用)
train_file_path = "SST-2/train.tsv"
try:
    train_df_sft = pd.read_csv(train_file_path, sep='\t', header=0)
except FileNotFoundError:
    print(f"エラー: {train_file_path} が見つかりません。")
    exit()

# ラベルをテキストに変換 (0 -> "negative", 1 -> "positive")
id_to_label_text = {0: "negative", 1: "positive"}
train_df_sft['label_text'] = train_df_sft['label'].map(id_to_label_text)

# SFTTrainerが期待する形式にデータを整形 (例: "text" カラムにプロンプト+回答)
def format_sft_data(example):
    prompt_text = f"Classify the sentiment of the following movie review: {example['sentence']}"
    # SFTTrainerは通常、プロンプトとレスポンスを結合したテキストを期待します
    # TRLのSFTTrainerのドキュメントや例を参考に、最適なフォーマットを確認してください。
    # ここでは、プロンプトとラベルを別々に保持し、後で処理することも考えられます。
    # 簡単のため、ここでは text フィールドにプロンプトと回答を結合する例は省略し、
    # SFTTrainerの `dataset_text_field` や `formatting_func` を活用することを想定します。
    # もしくは、より明示的にプロンプトとレスポンスを分けるデータ構造を使うことも可能です。

    # TRL SFTTrainerの簡単な使い方として、単一のテキストフィールドに instruction + output を入れる方法があります。
    # あるいは、formatting_func を使う方法もあります。
    # ここでは、SFTTrainerのformatting_funcを使うことを念頭に、必要な情報を保持しておきます。
    return {
        "prompt": f"Classify the sentiment of the following movie review: {example['sentence']}",
        "completion": example['label_text']
    }

# DataFrameをDatasetオブジェクトに変換
dataset_sft_train = Dataset.from_pandas(train_df_sft)
dataset_sft_train = dataset_sft_train.map(format_sft_data, remove_columns=train_df_sft.columns.tolist())

# SFTTrainerのためのフォーマット関数 (オプション)
# SFTTrainerは、Dataset内のテキストをどのようにプロンプトと回答として解釈するかを指定する
# formatting_func を受け入れることができます。
def formatting_prompts_func(example):
    # exampleはデータセットの一つの要素（辞書）
    # この関数は、プロンプトと回答を含む文字列のリストを返す
    # ここでは、各サンプルが 'prompt' と 'completion' キーを持つと仮定
    output_texts = []
    for i in range(len(example['prompt'])):
        text = f"Prompt: {example['prompt'][i]}\nCompletion: {example['completion'][i]}"
        output_texts.append(text)
    return output_texts

sft_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_SFT)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 前のステップで定義済み想定
sft_model.to(device)


sft_training_args = TrainingArguments(
    output_dir="./sft_flan_t5_sst2",
    num_train_epochs=1, # 例: SFTは1-3エポック程度
    per_device_train_batch_size=8,
    logging_steps=50,
    save_strategy="epoch",
    report_to="none", # wandbなどのロギングを無効化
)

# SFTTrainerの初期化 (dataset_text_fieldを指定する場合)
# まずは 'text' という結合済みフィールドを作る必要がある
# dataset_sft_train_formatted = dataset_sft_train.map(lambda x: {"text": f"Prompt: {x['prompt']}\nCompletion: {x['completion']}"})
# sft_trainer = SFTTrainer(
# model=sft_model,
# args=sft_training_args,
# train_dataset=dataset_sft_train_formatted,
# dataset_text_field="text", # "text" カラムに "Prompt: ... \nCompletion: ..." が入っている想定
# tokenizer=tokenizer,
# max_seq_length=256, # プロンプトと回答の合計の最大長
# )

# SFTTrainerの初期化 (formatting_funcを使う場合 - TRL 0.8.0以降など)
# この場合、train_datasetは 'prompt' と 'completion' を含む元の形式で良い
# SFTTrainerがformatting_funcを使って内部で文字列を組み立てる
# ただし、SFTTrainerのAPIのバージョンによって最適な方法が異なる場合があるので注意
# 最新のTRLでは、SFTTrainerはより柔軟なデータ形式を受け入れることがあります。
# ここでは、train_datasetが "prompt" と "completion" 列を持つとして進めます。
# (TRLのバージョンによっては、SFTTrainerが直接これらの列を認識しないかもしれません。
# その場合は、SFTConfigとpackingオプション、またはformatting_funcを適切に使う必要があります)

# TRLのドキュメントや最新の例を参照し、SFTTrainerへのデータの渡し方を調整してください。
# 簡単のため、ここでは train_dataset が SFTTrainer が期待する形式 (例: 'text' 列に結合済み文字列)
# になっているか、適切な formatting_func を渡す必要があるとします。
# 以下は概念的なコードです。
print("SFTモデルの訓練を開始します...")
# sft_trainer.train() # 実際の訓練コード (データ形式の整合性が取れている前提)
print("SFTモデルの訓練完了。（このステップは概念的なため、実際の訓練は省略しています）")
# sft_model.save_pretrained("./sft_flan_t5_sst2_model") # SFTモデルの保存
# tokenizer.save_pretrained("./sft_flan_t5_sst2_model")
# print("SFTモデルを './sft_flan_t5_sst2_model' に保存しました。")

# SFTモデルが準備できたと仮定して次に進みます。
# 実際には上記の訓練を実行し、保存/ロードするか、sft_model変数をDPOに渡します。
# ここでは、sft_model がファインチューニングされたものとして扱います。
# 例として、再度ベースモデルをロードしてSFT済みモデルの代わりとします（実際のシナリオではSFT済みモデルを使用）。
sft_model_for_dpo = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_SFT) # 本来は訓練済みのSFTモデル
sft_model_for_dpo.to(device)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

SFTモデルの訓練を開始します...
SFTモデルの訓練完了。（このステップは概念的なため、実際の訓練は省略しています）


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
from trl import DPOTrainer, DPOConfig

# DPO用のデータ準備 (SST-2の訓練データを使用)
# train_df_sft には 'sentence' と 'label_text' (正解ラベル) がある想定
def create_dpo_preference_data(example):
    prompt = f"Classify the sentiment of the following movie review: {example['sentence']}"
    chosen_completion = example['label_text']
    rejected_completion = "negative" if chosen_completion == "positive" else "positive"
    return {
        "prompt": prompt,
        "chosen": chosen_completion,
        "rejected": rejected_completion
    }

dpo_dataset = Dataset.from_pandas(train_df_sft) # train_df_sftを使用
dpo_dataset = dpo_dataset.map(create_dpo_preference_data, remove_columns=train_df_sft.columns.tolist())

print(f"\nDPO用データセットの例 (最初の1件):")
print(dpo_dataset[0])

# DPOConfig または TrainingArguments をDPO用に設定
# TRLのバージョンによってDPOConfigかTrainingArgumentsのどちらを使うかが変わることがあります
# ここでは TrainingArguments を流用できるケースを想定 (DPOTrainerが内部で処理)
# betaはDPOの重要なハイパーパラメータ (通常0.1-0.5程度)
dpo_training_args = TrainingArguments(
    output_dir="./dpo_flan_t5_sst2",
    num_train_epochs=1, # DPOはSFTより少ないエポックで済むことが多い
    per_device_train_batch_size=4, # DPOはメモリ消費が大きいことがある
    gradient_accumulation_steps=4,
    learning_rate=1e-5, # DPOではSFTより小さい学習率が推奨されることがある
    logging_steps=20,
    save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False, # DPOTrainerが "prompt", "chosen", "rejected" を使うため
    #beta=0.1, # DPOConfigで設定する場合。TrainingArgumentsに直接ない場合はDPOConfig経由
)

# DPOTrainerの初期化
# policyモデル (訓練対象) と ref_model (参照用、固定) を指定
# ref_modelをNoneにすると、DPOTrainerが内部でpolicyモデルのコピーをref_modelとして初期化時に作成します
dpo_trainer = DPOTrainer(
    model=sft_model_for_dpo,  # SFT済みモデルをポリシーモデルとして訓練
    ref_model=None,          # Noneにすると訓練開始時にmodelのコピーが使われる
    args=dpo_training_args,
    #beta=0.1,                # DPOのβパラメータ
    train_dataset=dpo_dataset,
    tokenizer=tokenizer,
    max_length=256,          # プロンプト + 回答の合計の最大長
    max_prompt_length=192,   # プロンプトの最大長
    # max_target_length=64, # 回答の最大長 (DPOTrainerのバージョンによる)
)

print("\nDPOモデルの訓練を開始します...")
# dpo_trainer.train() # 実際の訓練コード
print("DPOモデルの訓練完了。（このステップは概念的なため、実際の訓練は省略しています）")
# dpo_trainer.save_model("./dpo_flan_t5_sst2_model") # DPOモデルの保存
# print("DPOモデルを './dpo_flan_t5_sst2_model' に保存しました。")

# DPOモデルでの推論（SFTモデルと同様の方法で可能）
# (例)
dpo_model_loaded = AutoModelForSeq2SeqLM.from_pretrained("./dpo_flan_t5_sst2_model")
dpo_model_loaded.to(device)

test_sentence = "This movie was absolutely fantastic and inspiring!"
prompt_text = f"Classify the sentiment of the following movie review: {test_sentence}"
inputs = tokenizer(prompt_text, return_tensors="pt", padding=True, truncation=True, max_length=192).to(device)

dpo_model_loaded.eval()
with torch.no_grad():
  outputs = dpo_model_loaded.generate(**inputs, max_new_tokens=10) # max_new_tokens は "positive" 等のラベル長に合わせる
  predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(f"\n--- DPOモデルによる予測例 ---")
  print(f"テスト文: {test_sentence}")
  print(f"予測ラベルテキスト: {predicted_text}")

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]


DPO用データセットの例 (最初の1件):
{'prompt': 'Classify the sentiment of the following movie review: hide new secretions from the parental units ', 'chosen': 'negative', 'rejected': 'positive'}


TypeError: DPOTrainer.__init__() got an unexpected keyword argument 'tokenizer'